In [22]:
import os
import pandas as pd
from datetime import datetime as dt
import time
import plotly.express as px
import mysql.connector
from mysql.connector import Error
import numpy as np
import config

def create_chart_df(frame):
    times = []
    for k,v in frame.iterrows():    
        time = pd.date_range(v.start_time, v.end_time, freq="60min",).time
        times.append([v.boat_length,v.provider,v.date,v.dockage,time])

    df= pd.DataFrame(times,columns=['boat_length','provider','date','dockage','hour'])
    df2 = df.explode(column='hour')
    df2['hour'] = df2.hour.apply(lambda x: x.strftime('%H:%M %p'))
    df2.reset_index(inplace=True)
    df3 = pd.DataFrame(df2.groupby(['provider','hour','date'])['boat_length'].sum())
    df3['dockage'] = df2.dockage.iloc[1]
    df3['availability']= df3.dockage - df3.boat_length
    return df3

host = config.host
database = config.database
user = config.user
password = config.password

conn = mysql.connector.connect(host = host, database = database, user = user, password = password)

appointments = "SELECT \
    wp_jet_appointments.ID, \
    wp_jet_appointments.status, \
    a.post_title as service, \
    b.post_title as provider, \
    d.meta_value as dockage, \
    wp_jet_appointments.user_email, \
    wp_jet_appointments.date, \
    wp_jet_appointments.slot AS start_time, \
    wp_jet_appointments.slot_end AS end_time, \
    wp_jet_appointments.slot_end - wp_jet_appointments.slot as duration, \
    wp_jet_appointments.order_id AS confirmation, \
    u.display_name as captain, \
    c.meta_value AS boat_length, \
    wp_jet_appointments.phone, \
    wp_jet_appointments.comments \
FROM wp_jet_appointments \
INNER JOIN wp_posts AS a ON wp_jet_appointments.service = a.ID \
INNER JOIN wp_posts AS b ON wp_jet_appointments.provider = b.ID \
INNER JOIN wp_postmeta AS d ON wp_jet_appointments.provider = d.post_id \
INNER JOIN wp_users AS u ON wp_jet_appointments.user_id = u.ID \
INNER JOIN wp_usermeta AS c ON u.ID = c.user_id \
WHERE order_id > 0 \
AND d.meta_key = 'dock_length' \
AND c.meta_key = 'length' \
ORDER BY date, start_time ASC"

ap_data = pd.read_sql_query(appointments, conn)
ap_data.to_csv("assets/data/wp_jet_appointments.csv", index=False)
ap = pd.read_csv("assets/data/wp_jet_appointments.csv")

# Using a unix epoch time
ap["date"] = pd.to_datetime(ap["date"], unit='s')
ap["start_time"] = pd.to_datetime(ap["start_time"], unit='s').dt.time
ap["end_time"] = pd.to_datetime(ap["end_time"], unit='s').dt.time
ap["duration"] = pd.to_datetime(ap["duration"], unit='s').dt.time
ap["start_time"] = ap["start_time"].apply(lambda x: x.strftime('%I:%M %p'))
ap["end_time"] = ap["end_time"].apply(lambda x: x.strftime('%I:%M %p'))
dock_data = ap[["provider", "dockage", "date", "start_time", "end_time", "duration", "boat_length"]]

############################################################################################################

date = "2020-07-28"
dock_data = dock_data[dock_data.date==date]
try:
    df = create_chart_df(dock_data)
    df.to_csv('output.csv')
    now = dt.now().strftime("%a, %b %d, %Y, %H:%M:%S")
    print(f"{date}".upper())
    print()
    ind = df.index.get_level_values('provider').unique().tolist()
    for i in ind:
        print(i.upper())
        print(df.loc[i])
        print()
        print()
        print()
except(IndexError):
    print('WARNING'*500)
    print(f'No Data for {date}')

2020-07-28

PIZZERIA PORTOFINO
                     boat_length  dockage  availability
hour     date                                          
20:00 PM 2020-07-28           75      120            45
21:00 PM 2020-07-28           75      120            45





# DatePicker

In [53]:
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime as dt
x = dt.now()
datepicker = widgets.DatePicker(
    description='Pick a Date',
    disabled=False)
# slider = widgets.IntSlider()
datepicker.value = x
def on_change(v):
    x = v['new'] 
datepicker.observe(on_change, names='value')

display(datepicker)


DatePicker(value=datetime.datetime(2020, 7, 15, 21, 44, 56, 316475), description='Pick a Date')

In [54]:
print(datepicker.value)

2020-07-15 21:44:56.316475


In [55]:

from IPython.display import display
button = widgets.Button(description="Submit")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Submit', style=ButtonStyle())

Output()

Output()


# Feature columns and renaming

In [ ]:
dock_data = dock_data[["provider", "dockage", "date", "slot", "slot_end", "duration", "meta_value"]]
dock_data = dock_data.rename(columns={"slot":"start_time", "slot_end": "end_time","meta_value": "boat_length"})

In [ ]:
cw = dock_data.loc[dock_data["provider"] == "City Winery at the Chicago Riverwalk"]
cw.head()

create_chart_data

In [ ]:
# %%writefile chart_helper.py
def create_chart_df(frame):
    times = []
    for k,v in frame.iterrows():    
        time = pd.date_range(v.start_time, v.end_time, freq="60min",).time
        times.append([v.boat_length,v.provider,v.date,v.dockage,time])

    df= pd.DataFrame(times,columns=['boat_length','provider','date','dockage','hour'])
    df2 = df.explode(column='hour')
    df2['hour'] = df2.hour.apply(lambda x: x.strftime('%H:%M %p'))
    df2.reset_index(inplace=True)
    df3 = pd.DataFrame(df2.groupby(['provider','hour'])['boat_length'].sum())
    df3['dockage'] = df2.dockage.iloc[1]
    df3['availability']= df3.dockage - df3.boat_length
    return df3

In [ ]:
dockspace={'City Winery at the Chicago Riverwalk': 210,
 'Marina City': 55,
 'Pizzeria Portofino': 120}

df = create_chart_df(dock_data)

for key,value in dockspace.items():
    df.loc[key,'dockage'] = value

df.to_csv('output.csv')

In [86]:
print(dt.now().strftime("%a, %b %d, %Y, %H:%M:%S"))

Wed, Jul 15, 2020, 22:05:58


In [ ]:
dt.now().strftime

In [ ]:
dt.now().strftime

In [74]:
dt.now().strftime('%%l:%M %p')

' 9:59 PM'

In [106]:
# %%writefile update.py
import os
import pandas as pd
from datetime import datetime as dt
import time
import plotly.express as px
import mysql.connector
from mysql.connector import Error
import numpy as np
import config

def create_chart_df(frame):
    times = []
    for k,v in frame.iterrows():    
        time = pd.date_range(v.start_time, v.end_time, freq="60min",).time
        times.append([v.boat_length,v.provider,v.date,v.dockage,time])

    df= pd.DataFrame(times,columns=['boat_length','provider','date','dockage','hour'])
    df2 = df.explode(column='hour')
    df2['hour'] = df2.hour.apply(lambda x: x.strftime('%H:%M %p'))
    df2.reset_index(inplace=True)
    df3 = pd.DataFrame(df2.groupby(['provider','hour'])['boat_length'].sum())
    df3['dockage'] = df2.dockage.iloc[1]
    df3['availability']= df3.dockage - df3.boat_length
    return df3

date = input("What date do you want data for? 2020-05-14")

while True:
    host = config.host
    database = config.database
    user = config.user
    password = config.password

    conn = mysql.connector.connect(host = host, database = database, user = user, password = password)



    appointments = "SELECT user_id, status, service, provider, user_email, date, slot, slot_end, order_id \
                    FROM wp_jet_appointments \
                    ORDER BY slot asc"


    ap_data = pd.read_sql_query(appointments, conn)
    ap_data.to_csv("assets/data/wp_jet_appointments.csv", index=False)
    ap = pd.read_csv("assets/data/wp_jet_appointments.csv")
    duration = pd.to_datetime((ap.slot_end - ap.slot), unit='s').dt.time
    ap["duration"] = duration

    # Using a unix epoch time
    ap["date"] = pd.to_datetime(ap["date"], unit='s')
    ap["slot"] = pd.to_datetime(ap["slot"], unit='s').dt.time
    ap["slot_end"] = pd.to_datetime(ap["slot_end"], unit='s').dt.time
    ap["slot"] = ap["slot"].apply(lambda x: x.strftime('%I:%M %p'))
    ap["slot_end"] = ap["slot_end"].apply(lambda x: x.strftime('%I:%M %p'))
    ap["provider"] = ap["provider"].replace(415, "City Winery at the Chicago Riverwalk")
    ap["provider"] = ap["provider"].replace(417, "Marina City")
    ap["provider"] = ap["provider"].replace(421, "Pizzeria Portofino")
    ap["service"] = ap["service"].replace(478, "Dockage Under 24' 1hr")
    ap["service"] = ap["service"].replace(476, "Dockage Under 24' 2hr")
    ap["service"] = ap["service"].replace(474, "Dockage Under 24' 3hr")
    ap["service"] = ap["service"].replace(472, "Dockage 25' to 34' 1hr")
    ap["service"] = ap["service"].replace(470, "Dockage 25' to 34' 2hr")
    ap["service"] = ap["service"].replace(468, "Dockage 25' to 34' 3hr")
    ap["service"] = ap["service"].replace(466, "Dockage 35' to 44' 1hr")
    ap["service"] = ap["service"].replace(464, "Dockage 35' to 44' 2hr")
    ap["service"] = ap["service"].replace(462, "Dockage 35' to 44' 3hr")
    ap["service"] = ap["service"].replace(460, "Dockage 45' to 54' 1hr")
    ap["service"] = ap["service"].replace(458, "Dockage 45' to 54' 2hr")
    ap["service"] = ap["service"].replace(456, "Dockage 45' to 54' 3hr")
    ap["service"] = ap["service"].replace(454, "Dockage 55' to 64' 1hr")
    ap["service"] = ap["service"].replace(452, "Dockage 55' to 64' 2hr")
    ap["service"] = ap["service"].replace(450, "Dockage 55' to 64' 3hr")
    ap["service"] = ap["service"].replace(448, "Dockage 65' to 124' 1hr")
    ap["service"] = ap["service"].replace(446, "Dockage 65' to 124' 2hr")
    ap["service"] = ap["service"].replace(444, "Dockage 65' to 124' 3hr")
    ap["service"] = ap["service"].replace(442, "Dockage 125' and more 1hr")
    ap["service"] = ap["service"].replace(440, "Dockage 125' and more 2hr")
    ap["service"] = ap["service"].replace(438, "Dockage 125' and more 3hr")
    user_query = "SELECT * \
            FROM wp_usermeta \
            WHERE meta_key = 'length' "

    usermeta_data = pd.read_sql_query(user_query, conn)
    usermeta_data.to_csv("assets/data/wp_usermeta.csv", index=False)
    user = pd.read_csv("assets/data/wp_usermeta.csv")
    ap_m = pd.merge(ap, user, how="inner", on="user_id")
    pro_doc = {
        "provider" : ["City Winery at the Chicago Riverwalk", "Marina City", "Pizzeria Portofino"],
        "dockage" : [210, 55, 120]
        }

    pro_doc = pd.DataFrame(pro_doc)
    dock_data = pd.merge(pro_doc, ap_m, how="inner", on="provider")
    dock_data = dock_data[["provider", "dockage", "date", "slot", "slot_end", "duration", "meta_value"]]
    dock_data = dock_data[dock_data.date==date]
    dock_data = dock_data.rename(columns={"slot":"start_time", "slot_end": "end_time","meta_value": "boat_length"})

    df = create_chart_df(dock_data)
    df.to_csv('output.csv')
    now = dt.now().strftime("%a, %b %d, %Y, %H:%M:%S")
    print(f"Updated at {now}")
    time.sleep(10)

# df.to_csv('output.csv')

Overwriting update.py


In [101]:
date = input("What date do you want data for? 2020-05-14")

What date do you want data for? 2020-05-14'2020-06-11'


In [105]:
create_chart_df(test)

boat_length  dockage  \
provider                             hour                             
City Winery at the Chicago Riverwalk 13:00 PM           75      210   
                                     14:00 PM          225      210   
                                     15:00 PM          150      210   
                                     16:00 PM           75      210   
                                     19:00 PM           75      210   
                                     20:00 PM           75      210   
Marina City                          13:00 PM           75      210   
                                     14:00 PM           75      210   
                                     15:00 PM           75      210   
                                     16:00 PM           75      210   

                                               availability  
provider                             hour                    
City Winery at the Chicago Riverwalk 13:00 PM           135  
                                     14:00 PM           -15  
                                     15:00 PM            60  
                                     16:00 PM           135  
                                     19:00 PM           135  
                                     20:00 PM           135  
Marina City                          13:00 PM           135  
                                     14:00 PM           135  
                                     15:00 PM           135  
                                     16:00 PM           135

In [88]:
now = dt.now().strftime("%a, %b %d, %Y, %H:%M:%S")

'Wed, Jul 15, 2020, 22:07:26'

In [63]:
dock_data.groupby('provider').max()

,dockage,date,start_time,end_time,duration,boat_length
provider,,,,,,
City Winery at the Chicago Riverwalk,210,2020-07-25,12:00 PM,12:00 PM,02:00:00,124
Marina City,55,2020-06-11,01:00 PM,04:00 PM,03:00:00,75
Pizzeria Portofino,120,2020-08-29,12:00 PM,12:00 PM,02:00:00,75


In [98]:
output = pd.read_csv('output.csv'); output

,provider,hour,boat_length,dockage,availability
0,City Winery at the Chicago Riverwalk,11:00 AM,495,210,-285
1,City Winery at the Chicago Riverwalk,12:00 PM,570,210,-360
2,City Winery at the Chicago Riverwalk,13:00 PM,420,210,-210
3,City Winery at the Chicago Riverwalk,14:00 PM,574,210,-364
4,City Winery at the Chicago Riverwalk,15:00 PM,544,210,-334
5,City Winery at the Chicago Riverwalk,16:00 PM,195,210,15
6,City Winery at the Chicago Riverwalk,17:00 PM,120,210,90
7,City Winery at the Chicago Riverwalk,18:00 PM,45,210,165
8,City Winery at the Chicago Riverwalk,19:00 PM,420,210,-210
9,City Winery at the Chicago Riverwalk,20:00 PM,574,210,-364


# Bar Graph

In [ ]:
fig = px.bar(cw, x='start_time', y='boat_length',
             hover_data=['date', 'boat_length'], color='dockage',
             labels={'dockage':'Size Length Of Dock'}, height=400)
fig.write_html('dockage_availability.html', auto_open=True)

# Notes

### Data Fields
1. Unique User Id Per Boat
2. Boat Length
3. Dock Capacity
4. Status = Processed
5. G

In [ ]:
dock_data.columns

['provider', 'dockage', 'date', 'start_time', 'end_time', 'duration','boat_length']

hours = start_time + (duration - 1)
if hours > 1:
    for i in range(hours):
        slot = start_time + timedelta(i, hours)
        
        
   ['provider', 'dockage', 'date', 'start_time', 'end_time', 'duration','boat_length']